In [64]:
from data_loader import data_reader, colnames, colnames_bid
from matplotlib import pyplot as plt
import math
import pandas as pd

In [42]:
imp = data_reader('data\\imp.20131020.txt', colnames, 
                verbose=False, fuck_parsing=True)
bid = data_reader('data\\bid.20131020.txt', colnames_bid, verbose=False, fuck_parsing=True)
clk = data_reader('data\\clk.20131020.txt', colnames, verbose=False, fuck_parsing=True)
conv = data_reader('data\\conv.20131020.txt', colnames, verbose=False, fuck_parsing=True)


In [66]:
bid_rel = bid[['bid_id', 'timestamp']]
imp_rel = imp[['bid_id', 'paying_price']]
clk_rel = clk[['bid_id', 'paying_price']]
conv_rel = conv[['bid_id', 'paying_price']]
bids = bid_rel.merge(imp_rel,how='left', on='bid_id')
bids['not_bought'] = [math.isnan(price) for price in bids.paying_price]

,bid_id,timestamp,paying_price,not_bought
0,88db9ffd2dc45e3f77fbffe53d01d5a5,2013-10-20 10:00:00,11.0,False
1,67cf2695c255a3ed0ab22a384f054e3,2013-10-20 10:00:00,281.0,False
2,8a0576cbf105796609857de63277047,2013-10-20 10:00:00,11.0,False
3,ed8bec2da2b2ae4ada2a5183c7a48f49,2013-10-20 10:00:00,47.0,False
4,e1e94f59febbf30e35d46db440381c91,2013-10-20 10:00:00,5.0,False
5,e87b71afb493bd74b3093f7e56d0dd80,2013-10-20 10:00:00,38.0,False
6,78b53dd62dd860b07872a835ad6a438c,2013-10-20 10:00:00,198.0,False
7,e8b5db6185c55b5bea9edbd39d675575,2013-10-20 10:00:00,47.0,False
8,4289015cf8b2f8d7e74a8dc51ff2d755,2013-10-20 10:00:00,5.0,False
9,776c62a4410ec52c7508bed0765ec94a,2013-10-20 10:00:00,53.0,False


In [25]:

G = 1e5
B = 6e6
target_price =  B/G
subset_test


,bid_id,timestamp,log_type,ipinyou_id,user_agent,ip_address,region_id,city_id,ad_exchange,domain,...,ad_slot_height,ad_slot_visibility,ad_slot,ad_slot_floor_price,creative_id,bidding_price,paying_price,key_page_url,advertiser_id,user_tags
157052,ceeab9bb1fec9845ef3096b0abfdea7,2013-10-20 00:01:02,1,DABD6sF4xQ5,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,113.103.141.*,216,217,2,1bd36eadfdf00c75951d417a0e9dcd4,...,250,FirstView,Na,5,7323,277,120,null,2259,"[10684, 10102, 10006, 10024, 10145, 10114, 134..."
133823,ebfacbbead851896ed16699d30897238,2013-10-20 00:01:02,1,DAK012FAw~S,Mozilla/4.0 (compatible; MSIE 8.0; Windows NT ...,118.196.196.*,216,217,1,5777beaf611cab0f34afde6b9c3668c1,...,250,ThirdView,Na,0,7323,294,46,null,2259,None
20953,b09cd722f129dacec849a8738984852a,2013-10-20 00:01:03,1,D8VHIW5lx_6,Mozilla/5.0 (compatible; MSIE 9.0; Windows NT ...,119.123.104.*,216,219,1,20fc675468712705dbf5d3eda94126da,...,250,FourthView,Na,0,7323,294,201,null,2259,"[16661, 10684, 14273, 11576, 13042, 10102, 100..."
169113,f7812648e13bb35faef0d1fbe89d6cfe,2013-10-20 00:01:03,1,DACLUTBSyU7,Mozilla/5.0 (Windows NT 6.3; Trident/7.0; rv:1...,27.36.111.*,216,222,2,cbf15c0d399e933bf18bda41481b4658,...,60,FirstView,Na,5,7328,277,33,null,2259,"[10684, 13042, 10006, 10110, 10031, 16753]"
86233,a51fe544701af16c65d8b20747528898,2013-10-20 00:01:03,1,DAJNNa3vzeP,Mozilla/4.0 (compatible; MSIE 7.0; Windows NT ...,14.219.221.*,216,233,2,e4f50f61277983c1d78c0ce042e2514c,...,250,FirstView,Na,5,7323,277,7,null,2259,None
212589,55aae374565bcedac622d7f1842130fe,2013-10-20 00:01:03,1,DAFKWt3DvJv,Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.1...,211.162.62.*,216,217,1,704bdbbb6839e6bd83b7148ff8a0747b,...,250,ThirdView,Na,0,7321,294,30,null,2259,"[10063, 10116]"
22578,bdf5d07b7b2e73e9572cf7e6cee877dc,2013-10-20 00:01:03,1,DAK013EUw0U,Mozilla/4.0 (compatible; MSIE 6.0; Windows NT ...,113.106.102.*,216,220,2,ef5084da8be2863fedd2ceab7801410d,...,250,FirstView,Na,5,7323,277,144,null,2259,None
106167,9f5efdfd03734a5f6a11c37e1e3a011,2013-10-20 00:01:03,1,DAK013DDs3G,Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.1...,14.212.218.*,216,222,2,9f29c106b1894764ef88815ff2d43e2a,...,60,OtherView,Na,4,7328,277,4,null,2259,None
33731,3fb08c3438f0bbd4ca8c34942b6c3155,2013-10-20 00:01:03,1,DA5DlvENxWY,Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.1...,202.41.154.*,216,218,2,b3debf1265c0574133eeed3a657d4f9b,...,250,OtherView,Na,63,7321,277,82,null,2259,None
157839,e7c2efa2baaefd5a002102682a365ed6,2013-10-20 00:01:03,1,DAK013CbyUi,Mozilla/5.0 (compatible; MSIE 9.0; Windows NT ...,119.134.131.*,216,229,1,15904e7e858c438614266404a0b9e3dc,...,250,Na,Na,0,7321,294,44,null,2259,None
